In [1]:
import sys,os
import numpy as np

from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator

os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='12' # set number of MKL threads to run in parallel

In [13]:
L = 12
Jz, Jx, Jy, h = [-1, -1, -1, 0.01]

In [14]:
J_zz = [[1,i,(i+1)%L] for i in range(L)]
J_xy = [[1,i,(i+1)%L] for i in range(L)]
h_list = [[-1.0,i] for i in range(L)]
ops_dict = dict(Jz=[["zz", J_zz]], # J for z
                Jx=[["xx",J_xy]], # J for x and y
                Jy=[["yy",J_xy]],
                h=[["z",h_list]])
M_list = [[1.0/L,i] for i in range(L)]
G_list = [[1.0, i, j] for i in range(L) for j in range(L) if i != j]

basis_2d = spin_basis_1d(L ,pauli=False)
_H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
M = hamiltonian([["z",M_list]],[],basis=basis_2d, dtype=np.float64)
G = hamiltonian([["-+",G_list]],[],basis=basis_2d, dtype=np.float64)
params_dict=dict(Jz=Jz, Jx=Jx,Jy=Jy,h=h)

Hermiticity check passed!
Hermiticity check passed!
Hermiticity check passed!


/tmp/ipykernel_2159/1176320865.py:13: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_1d.spin.spin_basis_1d'>, to turn off this warning set check_symm=False in hamiltonian
  M = hamiltonian([["z",M_list]],[],basis=basis_2d, dtype=np.float64)
/tmp/ipykernel_2159/1176320865.py:14: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_1d.spin.spin_basis_1d'>, to turn off this warning set check_symm=False in hamiltonian
  G = hamiltonian([["-+",G_list]],[],basis=basis_2d, dtype=np.float64)


In [15]:
import cupy as cp

In [16]:
Hs = _H.tohamiltonian(params_dict)
H = Hs.toarray()
H_gpu = cp.asarray(H)

M_np = M.toarray()
M_gpu = cp.asarray(M_np)
G_gpu = cp.asarray(G.toarray())

In [17]:
E, V = cp.linalg.eigh(H_gpu)

In [18]:
T = 1
beta = 1/T
Z = cp.exp(-beta*E).sum()
M_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu) / Z
M2_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu@M_gpu) / Z
E_mean = (E*cp.exp(-beta*E)).sum() / Z
E2_mean = ((E*E)*cp.exp(-beta*E)).sum() / Z
G_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G_gpu) / Z

In [19]:
print(f"L = {L}", params_dict)
print(f"T               = {T}")
print(f"E               = {E_mean / L}")
print(f"C               = {(E2_mean - E_mean**2)*(beta**2)/L}")
print(f"M               = {M_mean}")
print(f"M^2             = {M2_mean}")
print(f"G               = {G_mean}")

L = 12 {'Jz': -1, 'Jx': -1, 'Jy': -1, 'h': 0.01}
T               = 1
E               = -0.13411824184551827
C               = 0.08393347654426941
M               = 0.0036824933153474
M^2             = 0.030699585335757243
G               = 2.8380576065773115


In [20]:
print(f"Chi             = {(0.0036824933153474)/0.01}")

Chi             = 0.36824933153474


In [255]:
2.8381884028754674 / 0.473485

5.99425198871235

In [266]:
2.3651511884691754 / 2.51318

0.9410990014520151

In [237]:
3.1803636899067547 / 0.636046

5.000210189053551

In [238]:
1.7311637729975817 / 0.345942

5.0042023605043084

In [219]:
3.1803636899067547 / 0.462785

6.872227254355164